In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import joblib

# ==================== DATA SCIENCE CORE ====================

SYMPTOMS = ["Fever", "Cough", "Headache", "Fatigue", "Nausea", "Shortness of breath"]
DISEASES = {
    "Common Cold": [1, 1, 0, 1, 0, 0],
    "Flu": [1, 1, 1, 1, 1, 0],
    "Migraine": [0, 0, 1, 1, 1, 0],
    "Pneumonia": [1, 1, 0, 1, 0, 1]  
}

# Prepare DataFrame
df = pd.DataFrame.from_dict(DISEASES, orient='index', columns=SYMPTOMS)
df['Disease'] = df.index

# Encode labels
le = LabelEncoder()
df['Disease_Encoded'] = le.fit_transform(df['Disease'])

# Train model
model = RandomForestClassifier()
model.fit(df[SYMPTOMS], df['Disease_Encoded'])

# Save models
joblib.dump(model, 'disease_predictor.joblib')
joblib.dump(le, 'label_encoder.joblib')

# Medical recommendations
RECOMMENDATIONS = {
    "Common Cold": {
        "doctor": "General Practitioner",
        "diet": "Warm fluids, vitamin C"
    },
    "Flu": {
        "doctor": "General Practitioner",
        "diet": "Hydration, antiviral foods"
    },
    "Migraine": {
        "doctor": "Neurologist",
        "diet": "Magnesium-rich foods"
    },
    "Pneumonia": {
        "doctor": "Pulmonologist",
        "diet": "Anti-inflammatory foods"
    }
}

# ==================== TKINTER GUI ====================
class SymptomCheckerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("MediGuide Symptom Checker")
        self.root.geometry("600x500")
        self.selected_symptoms = {symptom: 0 for symptom in SYMPTOMS}
        
        # Styling (preserved from your original)
        self.style = ttk.Style()
        self.style.configure('TFrame', background='#f0f8ff')
        self.style.configure('TLabel', background='#f0f8ff', font=('Arial', 10))
        self.style.configure('TButton', font=('Arial', 10), padding=5)
        self.style.map('TButton', 
                      foreground=[('active', 'black')], 
                      background=[('active', '#b0e0e6')])
        
        self.create_widgets()
        
        # Custom progressbar styles
        self.style.configure('green.Horizontal.TProgressbar', 
                           foreground='green', background='green')
        self.style.configure('yellow.Horizontal.TProgressbar', 
                           foreground='yellow', background='yellow')
        self.style.configure('red.Horizontal.TProgressbar', 
                           foreground='red', background='red')
    
    def create_widgets(self):
        # Main container
        main_frame = ttk.Frame(self.root, padding="10")
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Title
        ttk.Label(main_frame, text="Select Your Symptoms", 
                 font=('Arial', 14, 'bold')).pack(pady=10)
        
        # Symptoms checklist
        symptoms_frame = ttk.LabelFrame(main_frame, text="Symptoms", padding=10)
        symptoms_frame.pack(fill=tk.X, pady=10)
        
        for symptom in SYMPTOMS:
            var = tk.IntVar()
            ttk.Checkbutton(
                symptoms_frame, 
                text=symptom, 
                variable=var,
                command=lambda s=symptom, v=var: self.update_symptoms(s, v.get())
            ).pack(anchor=tk.W, padx=5, pady=2)
        
        # Analyze button
        ttk.Button(main_frame, 
                  text="Analyze Symptoms", 
                  command=self.analyze).pack(pady=20)
        
        # Results display
        self.results_frame = ttk.LabelFrame(main_frame, text="Results", padding=10)
        self.results_frame.pack(fill=tk.BOTH, expand=True)
        
        self.disease_label = ttk.Label(self.results_frame, text="", font=('Arial', 12))
        self.disease_label.pack(pady=5)
        
        self.doctor_label = ttk.Label(self.results_frame, text="")
        self.doctor_label.pack(pady=5)
        
        self.diet_label = ttk.Label(self.results_frame, text="")
        self.diet_label.pack(pady=5)
        
        # Confidence meter
        confidence_frame = ttk.Frame(self.results_frame)
        confidence_frame.pack(pady=10)
        
        ttk.Label(confidence_frame, text="Confidence: ").pack(side=tk.LEFT)
        self.confidence_meter = ttk.Progressbar(
            confidence_frame, 
            orient=tk.HORIZONTAL, 
            length=200, 
            mode='determinate'
        )
        self.confidence_meter.pack(side=tk.LEFT, padx=5)
        self.confidence_value = ttk.Label(confidence_frame, text="0%")
        self.confidence_value.pack(side=tk.LEFT)
        
        # Disclaimer
        ttk.Label(main_frame, 
                 text="Note: This is not a substitute for professional medical advice.",
                 font=('Arial', 8), 
                 foreground='red').pack(side=tk.BOTTOM, pady=5)
    
    def update_symptoms(self, symptom, value):
        self.selected_symptoms[symptom] = value
    
    def analyze(self):
        # Prepare input
        input_data = [self.selected_symptoms[s] for s in SYMPTOMS]
        input_df = pd.DataFrame([input_data], columns=SYMPTOMS)
        
        # Predict
        try:
            prediction = model.predict(input_df)[0]
            disease = le.inverse_transform([prediction])[0]
            proba = model.predict_proba(input_df)[0][prediction]
            
            # Update UI
            self.disease_label.config(text=f"Possible Condition: {disease}")
            self.doctor_label.config(text=f"Recommended Doctor: {RECOMMENDATIONS[disease]['doctor']}")
            self.diet_label.config(text=f"Dietary Advice: {RECOMMENDATIONS[disease]['diet']}")
            
            # Update confidence display
            confidence = int(proba * 100)
            self.confidence_meter['value'] = confidence
            self.confidence_value.config(text=f"{confidence}%")
            
            # Set confidence color
            if confidence > 70:
                self.confidence_meter['style'] = 'green.Horizontal.TProgressbar'
            elif confidence > 40:
                self.confidence_meter['style'] = 'yellow.Horizontal.TProgressbar'
            else:
                self.confidence_meter['style'] = 'red.Horizontal.TProgressbar'
                
        except Exception as e:
            messagebox.showerror("Error", f"Prediction failed: {str(e)}")

if __name__ == "__main__":
    root = tk.Tk()
    app = SymptomCheckerApp(root)
    root.mainloop()